最近[equinox](https://docs.kidger.site/equinox/)という[jax](https://jax.readthedocs.io/en/latest/)ベースの深層学習モデルを定義するライブラリを使ってみたのですが、これが中々いいと思ったので紹介ついでに強化学習してみます。他のjaxベースのライブラリにはDeepmindの[haiku]()やGoogle Researchの[flax]()があります。この2つは兄弟みたいなもので、実際のところあまり変わりはありません。というのも、jaxには試験的に書かれた[stax](https://jax.readthedocs.io/en/latest/jax.example_libraries.stax.html)というモデル定義用ライブラリがあり、haikuもflaxもstaxをベースにしているからです。equinoxのドキュメントにある[Compatibility with init-apply libraries](https://docs.kidger.site/equinox/examples/init_apply/)というページでは、これらのライブラリのやり方を「init-applyアプローチ」と呼んで軽く説明しているのですが、僕もこれについて簡単な説明を試みてみます。

# Jaxにできることとできないこと

そもそも、jaxというのは何をしてくれるライブラリなのでしょうか。ホームページのトップにはこう書かれています。

> JAX is Autograd and XLA, brought together for high-performance numerical computing.

[XLA](https://www.tensorflow.org/xla)というのは、Tensorflowのバックエンドとして開発された深層学習用の中間言語で、CPU/GPU用に数値計算コードを最適化してくれます。Jaxは、numpyライクなシンタックスのコードをXLAに変換することで、「numpyコードを高速にjitコンパイルすること」を可能にしています。では、Autogradというのはなんでしょうか？これは、jaxの開発者が以前に開発していたライブラリの名前でもありますが、自動微分全般のことを指すと考えていいでしょう。自分で勾配逆伝播のコードを書かなくても、jaxでは損失関数の各パラメタにおいての偏微分を勝手に計算してくれます。試しに、$f(x, y) = x^2 + y$の偏微分を計算してみましょう。

In [2]:
import jax
import jax.numpy as jnp

def f(x: jax.Array, y: jax.Array) -> jax.Array:
    return jnp.sum(x ** 2 + y)

x = jnp.array([3, 4, 5, 6, 7], dtype=jnp.float32)
y = jnp.array([5, 2, 5, 7, 2], dtype=jnp.float32)
jax.grad(f, argnums=(0, 1))(x, y)

(Array([ 6.,  8., 10., 12., 14.], dtype=float32),
 Array([1., 1., 1., 1., 1.], dtype=float32))



fのxでの偏微分は$2x$、yでの偏微分は$1$なので、正しく計算されているようですね。こんな感じで、jaxは自動的に偏微分を計算してくれるので、これをそのまま勾配降下法に使ってモデルを学習させることができます。深層ニューラルネットワークも結局パラメタと入力を引数にとる関数なので、全部のパラメタについて偏微分を計算してもらえば、それを使って学習できます。ついでに、この勾配をとる計算やモデルを更新する計算を`jax.jit`につっこめば、高速に計算してくれます。
ここで問題になるのは、深層ニューラルネットワークの場合パラメタが多すぎるので、こんな風にいちいち全てのパラメタについて変数を割り当ててプログラムを書いていたら大変すぎる、ということです。大変すぎる以外に何か問題はあるのかというと、特にないと思います。コードの再利用性くらいでしょうか。しかし、まあ面倒なものは面倒ですから、**パラメタを管理してくれる仕組み**がほしいなあ、と思うわけですね。

# init-applyアプローチによるパラメタ管理

stax, haiku, flaxでは、この「パラメタ管理の問題」を、「init-applyアプローチ」により解決しています。このアプローチは以下のようにまとめられます。
1. モデルはパラメータを持たない
2. モデルは`init`と`apply`の2つの関数を持つ
  - `init`は、入力例を受け取ってパラメタを初期化し、最初のパラメータを返す
  - `apply`は、入力とパラメタを受け取り、モデルの計算結果を返す
  
なので、flaxやhaikuのAPIは以下のような感じになります。flaxでは`__call__`を使うがhaikuはPyTorchと同じ`forward`を使う、`flax`のModuleは [`dataclasses.dataclass`](https://docs.python.org/ja/3/library/dataclasses.html#dataclasses.dataclass)デコレータにより定義されたクラスと同じような性質を持つなどの違いがありますが、まあそれくらいで、大して違いはないです。以下僕がflaxとhaikuの間をとって書いた適当な疑似コードです。
```python
class Linear(Module):
    def __call__(self, x):
        batch_size = x.shape[0]
        w = self.parameter(output, shape=(batch_size, self.output_size) init=self.w_init)
        b = self.parameter(output, shape=(1, self.output_size) init=self.w_init)
        return w @ x.T + b

model = Linear(output_size=10)
params = model.init(jnp.zeros(1, 100))
result = model.apply(params, jnp.zeros(10, 100))
```
こんな感じですかね。なお、上疑似コード中の`self.parameter`というメソッドは`flax`や`haiku`にある「パラメータをクラスに登録する機能」のことです。この機能により、各パラメタを値としてもつ`dict`を`init`により返すことができます。haikuの場合は、`params`は以下のような`dict`になっています。
```python
{
    "Linear/~/w": [...],
    "Linear/~/b": [...],
}
```

`stax`はただのreference implementationなのでこのような機能がなく、かわりに、数のレイヤーを組み合わせるコンビネーターを提供しています。ではこのアプローチにはどのようなメリット、デメリットがあるでしょうか。

**メリット**
- Moduleを初期化する際に入力されるArrayのshapeを指定しなくていい
  - Moduleの定義は`init`と`apply`という2つの関数を定義するだけで、パラメタは`init`が呼ばれた際に初期化されるため。
- 関数とデータを分離できる
  - Moduleは変更可能な変数を持たず、パラメタと完全に別に扱われるので、見通しがよくなる
- `init`、`apply`に`jit`や`vmap`などの関数デコレータを適用するコードが自然に書ける
  
**デメリット**
- Moduleは冗長
  - Moduleは「出力の次元」などの「モデルに関する設定」を持っているだけ
  - これらは`Config`など専用のクラスで管理されることが多く、`Config`と`Module`を両方持つのは冗長
- パラメタの要素のアクセスに型検査をするのが難しい
  - 例えばhaikuなら`params["Linear/~/w"]`のようにしてパラメタの各要素にアクセスするが、これは`pyright`や`mypy`のような静的型検査ツールにより検査されないので、実行時エラーを起こしやすい
- あまりオブジェクト指向的ではない
- (特にflax, haiku)仕組みがわかりにくい

こんなところでしょうか。

# equinoxの特徴

equinoxの特徴は、init-applyアプローチと異なり、「よりオブジェクト指向的な」（または、PyTorchに近い）インターフェースを志向している点にあります。
ドキュメントのトップページにある[Quick example](https://docs.kidger.site/equinox/#quick-example)を見てみましょう。

In [63]:
import equinox as eqx
import jax

class Linear(eqx.Module):
    weight: jax.Array
    bias: jax.Array

    def __init__(self, in_size, out_size, key):
        wkey, bkey = jax.random.split(key)
        self.weight = jax.random.normal(wkey, (out_size, in_size))
        self.bias = jax.random.normal(bkey, (out_size,))

    def __call__(self, x):
        return self.weight @ x + self.bias
    
@jax.jit
@jax.grad
def loss_fn(model, x, y):
    pred_y = jax.vmap(model)(x)
    return jax.numpy.mean((y - pred_y) ** 2)

batch_size, in_size, out_size = 32, 2, 3
model = Linear(in_size, out_size, key=jax.random.PRNGKey(0))
x = jax.numpy.zeros((batch_size, in_size))
y = jax.numpy.zeros((batch_size, out_size))
grads = loss_fn(model, x, y)
print("Model weight?", model.weight)
print("Grad?", grads)

Model weight? [[0.59902626 0.2172144 ]
 [0.660603   0.03266738]
 [1.2164948  1.1940813 ]]
Grad? Linear(weight=f32[3,2], bias=f32[3])


ここで、equinoxが他の「